# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments  \
0  Oct 31 2022 12:53PM  249717        10   MSP215366   
1  Oct 31 2022 12:53PM  249717        10   MSP215360   
2  Oct 31 2022 12:53PM  249717        10   MSP215372   
3  Oct 31 2022 12:53PM  249716        16     8699535   
4  Oct 31 2022 12:53PM  249716        16     8699542   
5  Oct 31 2022 12:53PM  249716        16     8704497   
6  Oct 31 2022 12:53PM  249715        10  0086131406   
7  Oct 31 2022 12:53PM  249715        10  0086131408   
8  Oct 31 2022 12:53PM  249715        10  0086131407   
9  Oct 31 2022 12:53PM  249715        10  0086131459   

                         Customer ShipmentStatus  
0                    Methapharm-G       Released  
1                Methapharm, Inc.       Released  
2                Methapharm, Inc.       Released  
3  Sartorius Bioprocess Solutions       Released  
4  Sartorius Bioprocess Solutions       Released  
5  Sartorius Bioprocess Solutions       Released  
6               ISDIN Corporation       Released  
7               ISDIN Corporation       Released  
8               ISDIN Corporation       Released  
9               ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
21  249713       Released          1
22  249714       Released          1
23  249715       Released         10
24  249716       Released          3
25  249717       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
249713                NaN        NaN       1.0
249714                NaN        NaN       1.0
249715                NaN        NaN      10.0
249716                NaN        NaN       3.0
249717                NaN        NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249679                1.0       29.0       1.0
249680                0.0        0.0       1.0
249684                0.0        0.0      11.0
249686                0.0        0.0       1.0
249692                0.0        2.0      17.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249679                  1         29         1
249680                  0          0         1
249684                  0          0        11
249686                  0          0         1
249692                  0          2        17

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               249679          1         29         1
1               249680          0          0         1
2               249684          0          0        11
3               249686          0          0         1
4               249692          0          2        17

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               249679         1        29        1
1               249680                            1
2               249684                           11
3               249686                            1
4               249692                   2       17

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                        Customer
0   Oct 31 2022 12:53PM  249717        10                    Methapharm-G
1   Oct 31 2022 12:53PM  249717        10                Methapharm, Inc.
3   Oct 31 2022 12:53PM  249716        16  Sartorius Bioprocess Solutions
6   Oct 31 2022 12:53PM  249715        10               ISDIN Corporation
16  Oct 31 2022 12:47PM  249714        10                  Virtue Rx, LLC
17  Oct 31 2022 12:43PM  249713        21               NBTY Global, Inc.
18  Oct 31 2022 12:41PM  249712        21               NBTY Global, Inc.
19  Oct 31 2022 12:38PM  249711        21               NBTY Global, Inc.
20  Oct 31 2022 12:35PM  249710        21               NBTY Global, Inc.
21  Oct 31 2022 12:34PM  249709        19                Eywa Pharma Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                        Customer  \
0  Oct 31 2022 12:53PM  249717        10                    Methapharm-G   
1  Oct 31 2022 12:53PM  249717        10                Methapharm, Inc.   
2  Oct 31 2022 12:53PM  249716        16  Sartorius Bioprocess Solutions   
3  Oct 31 2022 12:53PM  249715        10               ISDIN Corporation   
4  Oct 31 2022 12:47PM  249714        10                  Virtue Rx, LLC   
5  Oct 31 2022 12:43PM  249713        21               NBTY Global, Inc.   
6  Oct 31 2022 12:41PM  249712        21               NBTY Global, Inc.   
7  Oct 31 2022 12:38PM  249711        21               NBTY Global, Inc.   
8  Oct 31 2022 12:35PM  249710        21               NBTY Global, Inc.   
9  Oct 31 2022 12:34PM  249709        19                Eywa Pharma Inc.   

  Completed Executing Released  
0                            3  
1                            3  
2                            3  
3                           10  
4                            1  
5                            1  
6                            1  
7                            1  
8                            1  
9                            2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                        Customer  \
0  Oct 31 2022 12:53PM  249717        10                    Methapharm-G   
1  Oct 31 2022 12:53PM  249717        10                Methapharm, Inc.   
2  Oct 31 2022 12:53PM  249716        16  Sartorius Bioprocess Solutions   
3  Oct 31 2022 12:53PM  249715        10               ISDIN Corporation   
4  Oct 31 2022 12:47PM  249714        10                  Virtue Rx, LLC   
5  Oct 31 2022 12:43PM  249713        21               NBTY Global, Inc.   
6  Oct 31 2022 12:41PM  249712        21               NBTY Global, Inc.   
7  Oct 31 2022 12:38PM  249711        21               NBTY Global, Inc.   
8  Oct 31 2022 12:35PM  249710        21               NBTY Global, Inc.   
9  Oct 31 2022 12:34PM  249709        19                Eywa Pharma Inc.   

  Released Executing Completed  
0        3                      
1        3                      
2        3                      
3       10                      
4        1                      
5        1                      
6        1                      
7        1                      
8        1                      
9        2

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 31 2022 12:53PM  249717        10                    Methapharm-G   
1  Oct 31 2022 12:53PM  249717        10                Methapharm, Inc.   
2  Oct 31 2022 12:53PM  249716        16  Sartorius Bioprocess Solutions   
3  Oct 31 2022 12:53PM  249715        10               ISDIN Corporation   
4  Oct 31 2022 12:47PM  249714        10                  Virtue Rx, LLC   

  Released Executing Completed  
0        3                      
1        3                      
2        3                      
3       10                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 31 2022 12:53PM  249717        10                    Methapharm-G   
1  Oct 31 2022 12:53PM  249717        10                Methapharm, Inc.   
2  Oct 31 2022 12:53PM  249716        16  Sartorius Bioprocess Solutions   
3  Oct 31 2022 12:53PM  249715        10               ISDIN Corporation   
4  Oct 31 2022 12:47PM  249714        10                  Virtue Rx, LLC   

   Released  Executing  Completed  
0       3.0        NaN        NaN  
1       3.0        NaN        NaN  
2       3.0        NaN        NaN  
3      10.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                        Customer  \
0  Oct 31 2022 12:53PM  249717        10                    Methapharm-G   
1  Oct 31 2022 12:53PM  249717        10                Methapharm, Inc.   
2  Oct 31 2022 12:53PM  249716        16  Sartorius Bioprocess Solutions   
3  Oct 31 2022 12:53PM  249715        10               ISDIN Corporation   
4  Oct 31 2022 12:47PM  249714        10                  Virtue Rx, LLC   

   Released  Executing  Completed  
0       3.0        0.0        0.0  
1       3.0        0.0        0.0  
2       3.0        0.0        0.0  
3      10.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2247341      29.0       29.0        1.0
12          499412       1.0        1.0        0.0
15          249684      11.0        0.0        0.0
16          499409       4.0        0.0        0.0
19         1248497      22.0        2.0        0.0
20          249680       1.0        0.0        0.0
21          998846       4.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2247341      29.0       29.0        1.0
1        12   499412       1.0        1.0        0.0
2        15   249684      11.0        0.0        0.0
3        16   499409       4.0        0.0        0.0
4        19  1248497      22.0        2.0        0.0
5        20   249680       1.0        0.0        0.0
6        21   998846       4.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      29.0       29.0        1.0
1        12       1.0        1.0        0.0
2        15      11.0        0.0        0.0
3        16       4.0        0.0        0.0
4        19      22.0        2.0        0.0
5        20       1.0        0.0        0.0
6        21       4.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   29.0
1        12  Released    1.0
2        15  Released   11.0
3        16  Released    4.0
4        19  Released   22.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   16    19   20   21
Status                                         
Completed   1.0  0.0   0.0  0.0   0.0  0.0  0.0
Executing  29.0  1.0   0.0  0.0   2.0  0.0  0.0
Released   29.0  1.0  11.0  4.0  22.0  1.0  4.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   16    19   20   21
0          Completed   1.0  0.0   0.0  0.0   0.0  0.0  0.0
1          Executing  29.0  1.0   0.0  0.0   2.0  0.0  0.0
2           Released  29.0  1.0  11.0  4.0  22.0  1.0  4.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   16    19   20   21
0  Completed   1.0  0.0   0.0  0.0   0.0  0.0  0.0
1  Executing  29.0  1.0   0.0  0.0   2.0  0.0  0.0
2   Released  29.0  1.0  11.0  4.0  22.0  1.0  4.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()